In [65]:
from lib.TaxiEnv import TaxiEnv
import numpy as np
env = TaxiEnv()
ActionRow = []

env.print = False

# print(env.lastState)
# env.printCurState()
# (next_state, reward, done, info1, info2) = env.move(env.getRandomMove())
# print()
# print((next_state, reward, done, info1, info2))
# env.printCurState()
actionHopper = []

counter = 0
updated = 0
#Start by init of a 2D array with infinity values -> maximum cost
vzdalenosti = np.ones((5,5)) 
vzdalenosti.fill(float("inf"))

startState = env.getLastState()

actionMask = env.getLastActionMask()
print(actionMask)
akceId = 0
for i in actionMask:
            #If action is possible:
            if(i == 1):
                actionHopper.append((startState,akceId,0))
            akceId += 1
col, row = env.getPos(startState)
vzdalenosti[row,col] = 0

Target = 4

cilY, cilX = 0,0

env.printCurState()

while counter < 8:
    updated = 0
    newActionHopper = []
    for a in range(len(actionHopper)):        
        akce = actionHopper.pop(0)

        #Set the state we got from hopper:
        prevStav = akce[0]
        vzdalenost = akce[2]
        env.setState(prevStav)

        #Make a move:
        (next_state, reward, done, info1, info2) = env.move(akce[1])
        #Get possible actions in that state
        actionMask = info2["action_mask"]
        #If we got to current state more efficiently than ever before:
        col, row = env.getPos(next_state)
        if(vzdalenosti[row,col] > (vzdalenost- reward)):
            #print(reward)
            if(reward == 20):
                #print("yes")
                vzdalenosti[row,col] = vzdalenost-reward #float("-inf")
                cilY, cilX = row, col
            else:
                vzdalenosti[row,col] = vzdalenost-reward
            
            #Go through result and append all possible actions:
            akceId = 0
            for i in actionMask:
                #If action is possible:
                if(i == 1):
                    updated += 1
                    newActionHopper.append((next_state,akceId,vzdalenosti[row,col]))
                    if(akceId == Target):
                        vzdalenosti[row,col] = vzdalenost-reward #float("-inf")
                        cilY, cilX =  row,col
                akceId += 1
        else:
            #Otherwise, skip this step, we have been here
            continue
    #print(updated)
    #Make newActionHopper our current hopper
    #print(newActionHopper)
    actionHopper = newActionHopper
    #Just a debug counter
    counter += 1

print(vzdalenosti)


[1 1 1 1 0 0]
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


[[3. 2. 3. 4. 5.]
 [2. 1. 2. 3. 4.]
 [1. 0. 1. 2. 3.]
 [2. 1. 2. 3. 4.]
 [3. 2. 3. 4. 5.]]


In [66]:
kroky = []
kroky.append(4)
#cilY, cilX = np.unravel_index(vzdalenosti.argmin(),vzdalenosti.shape)
print(cilY)
print(cilX)

4
3


In [70]:

#PathFinding Itself

sample = np.ones((4,))
sample.fill(float("inf"))
bounds = vzdalenosti.shape
print(bounds)
#Pozor: inverzně indexy - běžně je to down, up, right, left. 
if((cilX + 1) < bounds[0] ):
    sample[0] = min(vzdalenosti[cilY,cilX],vzdalenosti[cilY,cilX+1])
if((cilY + 1) < bounds[1] ):
    sample[2] = min(vzdalenosti[cilY,cilX],vzdalenosti[cilY+1,cilX])
if((cilX - 1) >= 0 ):
    sample[1] = min(vzdalenosti[cilY,cilX],vzdalenosti[cilY,cilX-1])
if((cilY - 1)  >= 0 ):
    sample[3] = min(vzdalenosti[cilY,cilX],vzdalenosti[cilY-1,cilX])

print(sample)
#     sample = [vzdalenosti[cilX+1,cilY],vzdalenosti[cilX-1,cilY],vzdalenosti[cilX,cilY+1],vzdalenosti[cilX,cilY-1]]
krok = sample.argmin()

if(krok == 0):
    cilX += 1
    krok=1
elif(krok == 1):
    cilX -= 1
    krok=0
elif(krok == 2):
    cilY += 1
    krok=3
elif(krok == 3):
    cilY -= 1
    krok=2
kroky.append(krok)   
print(kroky)

if(0 in sample): 
    break
#env.move(krok)


(5, 5)
[1. 1. 1. 0.]
[4, 1, 1, 3, 3]


In [6]:
print(env.lastState)

(481, {'prob': 1.0, 'action_mask': array([0, 1, 0, 1, 0, 0], dtype=int8)})


In [33]:
env.printCurState()
env.setState(startState)
env.printCurState()


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

